In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2022-12-01"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

In [3]:
response = openai.Completion.create(
  engine=os.getenv('DEPLOYMENT_NAME'),
  prompt="Write a Kusto query to show name and resultCode columns from dependencies table.\nAnswer:  ",
   max_tokens=250,
  stop=["#",";"])

print(response.choices[0].text)

 dependencies 
| project name, resultCode


In [4]:
from azure.loganalytics import LogAnalyticsDataClient
from azure.loganalytics.models import QueryBody

workspace_id = 'a5a603ea-0972-4457-b753-56e44e8c8d80'
client = LogAnalyticsDataClient(credentials='YOUR_CREDENTIALS')

# query = "AzureActivity | summarize count() by bin(timestamp, 1h)"
query = '''
AmlDataSetEvent
| where split(OperationName, "/")[-1]=="READ" and AmlDatasetId !=""
| extend  Identity=(parse_json(Identity))
| project AmlDatasetId, UserName=Identity.UserName
| summarize  Count=count() by AmlDatasetId, UserName=tostring(UserName)'''
body = QueryBody(query=query)

response = client.query(workspace_id, body)
results = response.tables[0].rows

for result in results:
    print(result)

AttributeError: 'str' object has no attribute 'signed_session'

In [5]:
import os
import pandas as pd
from datetime import datetime, timezone
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

credential = DefaultAzureCredential()
client = LogsQueryClient(credential)

query = '''
AmlDataSetEvent
| where split(OperationName, "/")[-1]=="READ" and AmlDatasetId !=""
| extend  Identity=(parse_json(Identity))
| project AmlDatasetId, UserName=Identity.UserName
| summarize  Count=count() by AmlDatasetId, UserName=tostring(UserName)'''

start_time=datetime(2022, 10, 1, tzinfo=timezone.utc)
end_time=datetime(2023, 3, 10, tzinfo=timezone.utc)

try:
    response = client.query_workspace(
        workspace_id='a0083662-5f28-449c-9f7f-62ac8a3b7cc8',
        query=query,
        timespan=(start_time, end_time)
        )
    if response.status == LogsQueryStatus.PARTIAL:
        error = response.partial_error
        data = response.partial_data
        print(error)
    elif response.status == LogsQueryStatus.SUCCESS:
        data = response.tables
    for table in data:
        df = pd.DataFrame(data=table.rows, columns=table.columns)
        print(df)
except HttpResponseError as err:
    print("something fatal happened")
    print (err)

                                        AmlDatasetId       UserName  Count
0  azureml://locations/eastus2/workspaces/86d4aa3...  Louis Li (AI)      2
1               2f24db5e-dd88-4053-b7b1-f14784178136  Louis Li (AI)      1
2               a1bc0e1a-2b18-4243-8973-64eee4aa02a8  Louis Li (AI)      2
3               7a033689-4421-4d43-9210-a0a67da7b4df  Louis Li (AI)      2
4               62c10f96-4f9f-42ba-ab38-df10d8784bfa  Louis Li (AI)      1
5  azureml://locations/eastus2/workspaces/86d4aa3...  Louis Li (AI)      1


In [ ]:
from langchain.llms import AzureOpenAI
import os
from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from AzureOpenAIUtil.AzureCognitiveSearch import AzureCognitiveSearchWrapper
load_dotenv()
search = AzureCognitiveSearchWrapper(result_field_list=['highlights','article'], k=3)
# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.

llm = AzureOpenAI(temperature=0, deployment_name=os.getenv('DEPLOYMENT_NAME')) 
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description=''
    )
]

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="self-ask-with-search",  verbose=True)

# Now let's test it out!
result = agent.run("Who are top 2 competitors for Hillary Clinton in 2020 president election? Explain the reasons.")
print(result)